<a href="https://colab.research.google.com/github/sayakpaul/Adventures-in-TensorFlow-Lite/blob/master/Stable_Diffusion_to_TFLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [1]:
!pip install -U tf-nightly keras_cv -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.0/586.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.8/615.8 KB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.9/440.9 KB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 89.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 23.1.21 which is incompatible.


## Imports

In [1]:
import tensorflow as tf
import keras_cv
from tensorflow import keras

You do not have Waymo Open Dataset installed, so KerasCV Waymo metrics are not available.


In [2]:
print(f"TensorFlow version: {tf.__version__}")
print(f"KerasCV version: {keras_cv.__version__}")

TensorFlow version: 2.12.0-dev20230121
KerasCV version: 0.4.1


## Load Stable Diffusion

In [3]:
model = keras_cv.models.StableDiffusion(img_width=512, img_height=512)
diffusion_model = model.diffusion_model
text_encoder = model.text_encoder
decoder = model.decoder

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


## Conversions

In [4]:
converter_text = tf.lite.TFLiteConverter.from_keras_model(text_encoder)
converter_text.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_text_encoder = converter_text.convert()

with open("text_encoder.tflite", "wb") as f:
    f.write(tflite_text_encoder)

In [5]:
converter_decoder = tf.lite.TFLiteConverter.from_keras_model(decoder)
converter_decoder.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_decoder = converter_decoder.convert()

with open("image_decoder.tflite", "wb") as f:
    f.write(tflite_decoder)

In [6]:
converter_diffusion = tf.lite.TFLiteConverter.from_keras_model(diffusion_model)
converter_diffusion.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_diffusion = converter_diffusion.convert()

with open(f"diffusion_model.tflite", "wb") as f:
    f.write(tflite_diffusion)

In [7]:
!ls -lh *.tflite

-rw-r--r-- 1 root root 824M Jan 23 11:31 diffusion_model.tflite
-rw-r--r-- 1 root root  48M Jan 23 11:23 image_decoder.tflite
-rw-r--r-- 1 root root 118M Jan 23 11:22 text_encoder.tflite


## Inference

In [8]:
tokenizer = model.tokenizer

In [9]:
# Modified from 
# https://github.com/freedomtan/keras_cv_stable_diffusion_to_tflite/blob/main/text_to_image_using_converted_tflite_models_dynamic.ipynb
from keras_cv.models.stable_diffusion.constants import _UNCONDITIONAL_TOKENS
from keras_cv.models.stable_diffusion.constants import _ALPHAS_CUMPROD

import numpy as np
import math

class StableDiffusionTFLite:
    MAX_PROMPT_LENGTH = 77
    IMG_HEIGHT = 512
    IMG_WIDTH = 512
    
    def _get_initial_alphas(self, timesteps):
        alphas = [_ALPHAS_CUMPROD[t] for t in timesteps]
        alphas_prev = [1.0] + alphas[:-1]

        return alphas, alphas_prev
    
    def _get_initial_diffusion_noise(self, batch_size, seed):
        if seed is not None:
            return tf.random.stateless_normal(
                (batch_size, self.IMG_HEIGHT // 8, self.IMG_WIDTH // 8, 4),
                seed=[seed, seed],
            )
        else:
            return tf.random.normal(
                (batch_size, self.IMG_HEIGHT // 8, self.IMG_WIDTH // 8, 4)
            )
        
    def _get_timestep_embedding(self, timestep, batch_size, dim=320, max_period=10000):
        half = dim // 2
        freqs = tf.math.exp(
            -math.log(max_period) * tf.range(0, half, dtype=tf.float32) / half
        )
        args = tf.convert_to_tensor([timestep], dtype=tf.float32) * freqs
        embedding = tf.concat([tf.math.cos(args), tf.math.sin(args)], 0)
        embedding = tf.reshape(embedding, [1, -1])
        return tf.repeat(embedding, batch_size, axis=0)

    def _get_pos_ids(self):
        return tf.convert_to_tensor([list(range(self.MAX_PROMPT_LENGTH))], dtype=tf.int32)

    def encoded_token_padded(self, prompt):
        inputs = tokenizer.encode(prompt)
        phrase = inputs + [49407] * (self.MAX_PROMPT_LENGTH - len(inputs))
        phrase = tf.convert_to_tensor([phrase], dtype=tf.int32)

        return phrase, self._get_pos_ids()
    
    def encode_text(self, prompt):
        i_text_encoder = tf.lite.Interpreter('text_encoder.tflite')
        i_text_encoder.allocate_tensors()
        input_details = i_text_encoder.get_input_details()
        output_details = i_text_encoder.get_output_details()

        token, pos = self.encoded_token_padded(prompt)
        i_text_encoder.set_tensor(input_details[0]['index'], token)
        i_text_encoder.set_tensor(input_details[1]['index'], pos)

        i_text_encoder.invoke()

        output_data = i_text_encoder.get_tensor(output_details[0]['index'])

        return output_data
    
    def encode_text_2(self, token, pos):
        i_text_encoder = tf.lite.Interpreter('text_encoder.tflite')
        i_text_encoder.allocate_tensors()
        input_details = i_text_encoder.get_input_details()
        output_details = i_text_encoder.get_output_details()

        i_text_encoder.set_tensor(input_details[0]['index'], token)
        i_text_encoder.set_tensor(input_details[1]['index'], pos)

        i_text_encoder.invoke()

        output_data = i_text_encoder.get_tensor(output_details[0]['index'])

        return output_data


    def _expand_tensor(self, text_embedding, batch_size):
        """Extends a tensor by repeating it to fit the shape of the given batch size."""
        text_embedding = tf.squeeze(text_embedding)
        if text_embedding.shape.rank == 2:
            text_embedding = tf.repeat(
                tf.expand_dims(text_embedding, axis=0), batch_size, axis=0
            )
        return text_embedding
    
    def _get_unconditional_context(self):
        unconditional_tokens = tf.convert_to_tensor(
            [_UNCONDITIONAL_TOKENS], dtype=tf.int32
        )
        unconditional_context = self.encode_text_2(unconditional_tokens, self._get_pos_ids())

        return unconditional_context
    
    def get_index_of_name(self, tensors, name):
        for a in tensors:
            if a['name'] == name:
                return a['index']

    def diffusion_model(self, latent, t_emb, unconditional_context):
        i_diffusion = tf.lite.Interpreter('diffusion_model.tflite')
        # batch_size = latent.shape[0]

        i_diffusion_input_details = i_diffusion.get_input_details()
        i_diffusion_output_details = i_diffusion.get_output_details()

        i_diffusion.allocate_tensors()

        for i in range(len(i_diffusion_input_details)):
            if len(i_diffusion_input_details[i]['shape'].tolist()) == 4:
                # i_diffusion.resize_tensor_input(i, latent.get_shape().as_list())
                i_diffusion.set_tensor(i_diffusion_input_details[i]['index'], latent)
            elif len(i_diffusion_input_details[i]['shape'].tolist()) == 3:
                # i_diffusion.resize_tensor_input(i, unconditional_context.get_shape().as_list())
                i_diffusion.set_tensor(i_diffusion_input_details[i]['index'], unconditional_context)
            else:
                # i_diffusion.resize_tensor_input(i, t_emb.get_shape().as_list())
                i_diffusion.set_tensor(i_diffusion_input_details[i]['index'], t_emb)

        i_diffusion.invoke()
        
        output_data = i_diffusion.get_tensor(i_diffusion_output_details[0]['index'])
        return output_data
    
    def decode(self, encoded_image):
        i_decoder = tf.lite.Interpreter('image_decoder.tflite')
        i_decoder.allocate_tensors()
        input_details = i_decoder.get_input_details()
        output_details = i_decoder.get_output_details()

        i_decoder.set_tensor(input_details[0]['index'], encoded_image)

        i_decoder.invoke()

        output_data = i_decoder.get_tensor(output_details[0]['index'])

        return output_data

    def generate_image(
        self,
        encoded_text,
        negative_prompt=None,
        batch_size=1,
        num_steps=50,
        unconditional_guidance_scale=7.5,
        diffusion_noise=None,
        seed=None,
    ):
        context = self._expand_tensor(encoded_text, batch_size)

        if negative_prompt is None:
            unconditional_context = tf.repeat(
                self._get_unconditional_context(), batch_size, axis=0
            )
        else:
            unconditional_context = self.encode_text(negative_prompt)
            unconditional_context = self._expand_tensor(
                unconditional_context, batch_size
            )
        if diffusion_noise is not None:
            diffusion_noise = tf.squeeze(diffusion_noise)
            if diffusion_noise.shape.rank == 3:
                diffusion_noise = tf.repeat(
                    tf.expand_dims(diffusion_noise, axis=0), batch_size, axis=0
                )
            latent = diffusion_noise
        else:
            latent = self._get_initial_diffusion_noise(batch_size, seed)

        timesteps = tf.range(1, 1000, 1000 // num_steps)
        alphas, alphas_prev = self._get_initial_alphas(timesteps)
        progbar = keras.utils.Progbar(len(timesteps))
        iteration = 0
        for index, timestep in list(enumerate(timesteps))[::-1]:
            latent_prev = latent  # Set aside the previous latent vector
            t_emb = self._get_timestep_embedding(timestep, batch_size)
            unconditional_latent = self.diffusion_model(latent, t_emb, unconditional_context)
            
            latent = self.diffusion_model(latent, t_emb, context)
            latent = unconditional_latent + unconditional_guidance_scale * (
                latent - unconditional_latent
            )
            a_t, a_prev = alphas[index], alphas_prev[index]
            pred_x0 = (latent_prev - math.sqrt(1 - a_t) * latent) / math.sqrt(a_t)
            latent = latent * math.sqrt(1.0 - a_prev) + math.sqrt(a_prev) * pred_x0
            iteration += 1
            progbar.update(iteration)

        # Decoding stage
        decoded = self.decode(latent)
        decoded = ((decoded + 1) / 2) * 255
        return np.clip(decoded, 0, 255).astype("uint8")

In [ ]:
model_t = StableDiffusionTFLite()
encoded_text_tflite = model_t.encode_text('a photo of an astronaut riding a horse on Mars')
images = model_t.generate_image(encoded_text_tflite, num_steps=25) # Takes a while to generate (~30 mins)

 5/25 [=====>........................] - ETA: 23:44

In [ ]:
from PIL import Image 

for i in range(len(images)):
    image = Image.fromarray(images[i])
    image.save(f"{i}.png")